In [2]:
from __future__ import print_function
from builtins import range

"""
SECTION 1 : Load and setup data for training

the datasets separated in two files from originai datasets:
iris_train.csv = datasets for training purpose, 80% from the original data
iris_test.csv  = datasets for testing purpose, 20% from the original data
"""
import pandas as pd

#load
datatrain = pd.read_csv('../Datasets/iris/iris_train.csv')

#change string value to numeric
datatrain.loc[datatrain['species']=='Iris-setosa', 'species']=0
datatrain.loc[datatrain['species']=='Iris-versicolor', 'species']=1
datatrain.loc[datatrain['species']=='Iris-virginica', 'species']=2
datatrain = datatrain.apply(pd.to_numeric)

#change dataframe to array
datatrain_array = datatrain.values

#split x and y (feature and target)
xtrain = datatrain_array[:,:4]
ytrain = datatrain_array[:,4]

In [6]:
datatrain_array[:4]

array([[5.1, 3.4, 1.5, 0.2, 0. ],
       [6.3, 2.5, 5. , 1.9, 2. ],
       [5.9, 3. , 4.2, 1.5, 1. ],
       [5.7, 2.6, 3.5, 1. , 1. ]])

In [7]:
print(xtrain.shape)
print(ytrain.shape)

(120, 4)
(120,)


In [8]:
"""
SECTION 2 : Build and Train Model

Multilayer perceptron model, with one hidden layer.
input layer : 4 neuron, represents the feature of Iris
hidden layer : 10 neuron, activation using ReLU
output layer : 3 neuron, represents the class of Iris

optimizer = stochastic gradient descent with no batch-size
loss function = categorical cross entropy
learning rate = 0.01
epoch = 500
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.manual_seed(1234)

#hyperparameters
hl = 10
lr = 0.01
num_epoch = 500
num_feature=4
num_class=3

#build model
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_feature, hl)
        self.fc2 = nn.Linear(hl, num_class)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
net = Net()

#choose optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

#train
for epoch in range(num_epoch):
    X = Variable(torch.Tensor(xtrain).float())
    Y = Variable(torch.Tensor(ytrain).long())

    #feedforward - backprop
    optimizer.zero_grad()
    out = net(X)
    loss = criterion(out, Y)
    loss.backward()
    optimizer.step()

    if (epoch) % 50 == 0:
        print ('Epoch [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epoch, loss.item()))

Epoch [1/500] Loss: 1.2674
Epoch [51/500] Loss: 1.0102
Epoch [101/500] Loss: 0.8903
Epoch [151/500] Loss: 0.7676
Epoch [201/500] Loss: 0.6616
Epoch [251/500] Loss: 0.5830
Epoch [301/500] Loss: 0.5266
Epoch [351/500] Loss: 0.4841
Epoch [401/500] Loss: 0.4499
Epoch [451/500] Loss: 0.4205


In [1]:
"""
SECTION 3 : Testing model
"""
#load
datatest = pd.read_csv('../Datasets/iris/iris_test.csv')

#change string value to numeric
datatest.loc[datatest['species']=='Iris-setosa', 'species']=0
datatest.loc[datatest['species']=='Iris-versicolor', 'species']=1
datatest.loc[datatest['species']=='Iris-virginica', 'species']=2
datatest = datatest.apply(pd.to_numeric)

#change dataframe to array
datatest_array = datatest.values

#split x and y (feature and target)
xtest = datatest_array[:,:4]
ytest = datatest_array[:,4]

#get prediction
X = Variable(torch.Tensor(xtest).float())
Y = torch.Tensor(ytest).long()
out = net(X)
_, predicted = torch.max(out.data, 1)

#get accuration
print('Accuracy of the network %d %%' % (100 * torch.sum(Y==predicted) / 30))

Epoch [1/500] Loss: 1.2674
Epoch [51/500] Loss: 1.0102
Epoch [101/500] Loss: 0.8903
Epoch [151/500] Loss: 0.7676
Epoch [201/500] Loss: 0.6616
Epoch [251/500] Loss: 0.5830
Epoch [301/500] Loss: 0.5266
Epoch [351/500] Loss: 0.4841
Epoch [401/500] Loss: 0.4499
Epoch [451/500] Loss: 0.4205
Accuracy of the network 100 %
